# Lab 4 - Visión Por Computadora

Autores:

- Nelson García
- Joaquín Puente
- Diego Linares

---

## Task 1: 

##### 1. Una homografía 𝐻 es una matriz de 3 × 3. 

##### 1.1 Explique matemáticamente por qué, aunque tiene 9 elementos, solo posee 8 grados de libertad (GDL). 

##### 1.2 Adicionalmente, respoda. Si tuviéramos una cámara que solo rota sobre su eje óptico (sin traslación ni cambio de perspectiva), ¿la matriz de transformación sigue teniendo 8 GDL o se reduce? Demuestre la estructura de dicha matriz simplificada.


##### 2. En el algoritmo DLT (Direct Linear Transform), convertimos el problema 𝑥' = 𝐻𝑥 en un sistema de la forma 𝐴ℎ = 0. Explique por qué buscamos el vector singular asociado al menor valor singular de 𝐴 en lugar de simplemente invertir la matriz. ¿Qué representa geométricamente ese "menor valor singular" cuando los datos tienen ruido?


##### 3. Si usted selecciona 4 puntos para calcular 𝐻, pero 3 de ellos son colineales (están en la misma línearecta), el algoritmo fallará. Explique algebraicamente qué le sucede a la matriz 𝐴 del sistema DLT en este caso y por qué no tiene solución única.


---

## Task 2:

El objetivo de esta parte es implementar el pipeline de alineación sin depender de cajas negras. Por ello
considere que no deben de usar cv2.findHonography o cv2.RANSAC. Además para este laboratorio
necesitara crear su propio dataset, por ello tome 3 fotografías propias de una escena planar (i.e. una
pancarta en una pared, un cuadro, o una fachada de edificio lejana) con ángulos y perspectivas
drásticamente diferentes. Con esto realice:

1. Detección y Macheo

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


def imread_color(path: str):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(f"No pude leer la imagen: {path}")
    return img


def to_gray(img_bgr):
    return cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)


def create_feature(method: str):
    """
    method: 'sift' o 'orb'
    """
    method = method.lower()
    if method == "sift":
        # Requiere opencv-contrib-python
        if not hasattr(cv2, "SIFT_create"):
            raise RuntimeError(
                "Tu OpenCV no tiene SIFT_create. Instala opencv-contrib-python o usa ORB."
            )
        return cv2.SIFT_create()
    elif method == "orb":
        # ORB funciona con opencv-python
        return cv2.ORB_create(nfeatures=4000)
    else:
        raise ValueError("method debe ser 'sift' o 'orb'")


def create_bf(method: str):
    method = method.lower()
    if method == "sift":
        norm = cv2.NORM_L2
        return cv2.BFMatcher(normType=norm, crossCheck=False)
    else:
        norm = cv2.NORM_HAMMING
        return cv2.BFMatcher(normType=norm, crossCheck=False)


def draw_and_save_matches(img1_bgr, kp1, img2_bgr, kp2, matches, out_path, max_draw=600):
    """
    Dibuja una cantidad grande de matches para que se vea el ruido.
    """
    # Limitar solo para que la imagen no sea enorme
    matches_to_draw = matches[:max_draw] if len(matches) > max_draw else matches
    vis = cv2.drawMatches(
        img1_bgr, kp1, img2_bgr, kp2, matches_to_draw,
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    cv2.imwrite(out_path, vis)
    return vis


def show_bgr(img_bgr, title=""):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(14, 7))
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis("off")
    plt.show()


def match_pair(img1_path, img2_path, method="sift", out_dir="out",
               unfiltered_draw=800, filtered_draw=300, ratio=0.75):
    os.makedirs(out_dir, exist_ok=True)

    img1 = imread_color(img1_path)
    img2 = imread_color(img2_path)

    gray1 = to_gray(img1)
    gray2 = to_gray(img2)

    feat = create_feature(method)
    bf = create_bf(method)

    kp1, des1 = feat.detectAndCompute(gray1, None)
    kp2, des2 = feat.detectAndCompute(gray2, None)

    if des1 is None or des2 is None or len(kp1) < 10 or len(kp2) < 10:
        raise RuntimeError("Muy pocos keypoints/descriptores. Intenta ORB o mejor iluminación/escena.")

    # ---------------------------
    # (c) MATCHES ANTES DE FILTRAR (con ruido)
    # ---------------------------
    raw_matches = bf.match(des1, des2)  # fuerza bruta, 1-1 sin ratio test
    # OJO: ordenar por distancia NO es un “filtro” tipo RANSAC, solo orden para visualizar mejor
    raw_matches = sorted(raw_matches, key=lambda m: m.distance)

    out_unf = os.path.join(out_dir, f"unfiltered_{method}_{os.path.basename(img1_path)}__{os.path.basename(img2_path)}.png")
    vis_unf = draw_and_save_matches(img1, kp1, img2, kp2, raw_matches, out_unf, max_draw=unfiltered_draw)

    # ---------------------------
    # (opcional) FILTRADO CLÁSICO: Lowe ratio test (para comparar)
    # ---------------------------
    # Esto NO es RANSAC, solo limpia correspondencias ambiguas.
    knn = bf.knnMatch(des1, des2, k=2)
    good = []
    for m, n in knn:
        if m.distance < ratio * n.distance:
            good.append(m)
    good = sorted(good, key=lambda m: m.distance)

    out_f = os.path.join(out_dir, f"filtered_ratio_{method}_{os.path.basename(img1_path)}__{os.path.basename(img2_path)}.png")
    vis_f = draw_and_save_matches(img1, kp1, img2, kp2, good, out_f, max_draw=filtered_draw)

    print("====================================")
    print(f"Par: {img1_path}  <->  {img2_path}")
    print(f"Método: {method}")
    print(f"Keypoints: img1={len(kp1)} img2={len(kp2)}")
    print(f"Matches (sin filtrar): {len(raw_matches)}")
    print(f"Matches (ratio test):  {len(good)}")
    print(f"Guardado sin filtrar:  {out_unf}")
    print(f"Guardado filtrado:     {out_f}")
    print("====================================")

    # Mostrar en pantalla (opcional)
    show_bgr(vis_unf, title=f"UNFILTERED matches ({method})")
    show_bgr(vis_f, title=f"FILTERED (ratio test) matches ({method})")




In [ ]:

imgA = "img1.jpg"
imgB = "img2.jpg"
imgC = "img3.jpg"

# Prueba SIFT primero; si falla, cambia a 'orb'
method = "sift"  # o "orb"

# Puedes matchear por pares (A-B, B-C, A-C)
match_pair(imgA, imgB, method=method, out_dir="out", unfiltered_draw=900, filtered_draw=300, ratio=0.75)
match_pair(imgB, imgC, method=method, out_dir="out", unfiltered_draw=900, filtered_draw=300, ratio=0.75)
match_pair(imgA, imgC, method=method, out_dir="out", unfiltered_draw=900, filtered_draw=300, ratio=0.75)
